# asigna_estacion_incidentes_1.1

En este script se va a asignar a cada incidente la estacion mas cercana al lugar del mismo. El metodo va a ser cruzar ambos ficheros por fecha, estableciendo una serie de reestricciones, a saber:

    -temperatura media, maxima y minimas informadas.
    -precipitaciones informadas
    -presion informada
    -velocidad del viento informada

una vez hecho esto, lo que se hara es para cada incidente calcular la distancia a las diferentes estaciones, y agregar el registro seleccionando la distancia minima.

In [1]:
import pandas as pd
import numpy as np
from numpy import radians
from math import sin, cos, sqrt, atan2

In [2]:
df_meteo=pd.read_csv('../DATASETS/ESTACIONES/historico_estaciones_coordenadas_decimal.csv',sep='|',error_bad_lines=False)
df_meteo['Longitud_y']=radians(df_meteo[['Longitud_y']])
df_meteo['Latitud_y']=radians(df_meteo[['Latitud_y']])
df_meteo.head()

,Fecha,ID,Estacion,Estado_x,Latitud_y,Longitud_y,T. med. (ºC),T. max (ºC),T. min (ºC),Prec. (mm),Presion/ Geopot.,Dir. vi.,Vel. vi. (Km/h),Nub.,Prof. nieve (cm),Insolac. (horas)
0,31/01/2011,8001,LA CORUNA,SPAIN,0.756891,-0.146899,8.3,10.4,6.2,0.0,1024.1 Hpa,65º(NE),14,7/8,-,0.7
1,31/01/2011,8008,LUGO/ROZAS,SPAIN,0.752528,-0.130027,2.6,4.4,0.9,0.2,1024.3 Hpa,325º(NW),2,8/8,-,0.0
2,31/01/2011,8011,ASTURIAS/AVILES,SPAIN,0.760091,-0.105302,5.5,8.0,2.9,0.0,1025.0 Hpa,203º(SW),6,7/8,-,0.8
3,31/01/2011,8014,GIJON-MUSEL,SPAIN,0.760265,-0.099498,6.6,9.0,4.2,0.0,1024.1 Hpa,224º(SW),5,7/8,-,0.4
4,31/01/2011,8015,OVIEDO,SPAIN,0.756600,-0.102393,4.4,7.4,1.4,Ip,1024.5 Hpa,235º(SW),2,7/8,-,1.6


In [3]:
df_incidentes=pd.read_csv('../DATASETS/AHOGADOS/CSV/Datos_2019-2013.csv',sep='|',error_bad_lines=False)
df_incidentes['Longitud']=radians(df_incidentes[['Longitud']])
df_incidentes['Latitud']=radians(df_incidentes[['Latitud']])
df_incidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6482 entries, 0 to 6481
Data columns (total 26 columns):
Fecha              6482 non-null object
IdAhogado          6482 non-null int64
IdPersona          6482 non-null int64
Localidad          6482 non-null object
Provincia          6482 non-null object
CCAA               6482 non-null object
Hora               5381 non-null object
Latitud            6482 non-null float64
Longitud           6482 non-null float64
Sexo               6481 non-null object
Edad               5366 non-null float64
Nacionalidad       6482 non-null object
Origen             6482 non-null object
Extraccion         6482 non-null object
Titular            6482 non-null object
Causa              6482 non-null object
TipoAhogamiento    6482 non-null object
Factor             6482 non-null object
Intervencion       6482 non-null object
Pronostico         6482 non-null object
Localizacion       6482 non-null object
Riesgo             6482 non-null object
Reanimacion 

In [4]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359612 entries, 0 to 359611
Data columns (total 16 columns):
Fecha               359612 non-null object
ID                  359612 non-null int64
Estacion            359612 non-null object
Estado_x            359612 non-null object
Latitud_y           359612 non-null float64
Longitud_y          359612 non-null float64
T. med. (ºC)        359612 non-null object
T. max (ºC)         359612 non-null object
T. min (ºC)         359612 non-null object
Prec. (mm)          359612 non-null object
Presion/ Geopot.    359612 non-null object
Dir. vi.            359612 non-null object
Vel. vi. (Km/h)     359612 non-null object
Nub.                359612 non-null object
Prof. nieve (cm)    359612 non-null object
Insolac. (horas)    283707 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 43.9+ MB


In [5]:
df_meteo=df_meteo.rename(columns={"T. med. (ºC)": "T_med","T. max (ºC)": "T_max","T. min (ºC)": "T_min","Prec. (mm)": "Precipitaciones","Presion/ Geopot.": "Presion","Vel. vi. (Km/h)": "V_Viento"})


In [6]:
df_meteo=df_meteo[df_meteo.T_med != '-']
df_meteo=df_meteo[df_meteo.T_max != '-']
df_meteo=df_meteo[df_meteo.T_min != '-']
df_meteo=df_meteo[df_meteo.Precipitaciones != '-']
df_meteo=df_meteo[df_meteo.Presion != '-']
df_meteo=df_meteo[df_meteo.V_Viento != '-']

df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279224 entries, 0 to 359597
Data columns (total 16 columns):
Fecha               279224 non-null object
ID                  279224 non-null int64
Estacion            279224 non-null object
Estado_x            279224 non-null object
Latitud_y           279224 non-null float64
Longitud_y          279224 non-null float64
T_med               279224 non-null object
T_max               279224 non-null object
T_min               279224 non-null object
Precipitaciones     279224 non-null object
Presion             279224 non-null object
Dir. vi.            279224 non-null object
V_Viento            279224 non-null object
Nub.                279224 non-null object
Prof. nieve (cm)    279224 non-null object
Insolac. (horas)    214108 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 36.2+ MB


In [7]:
df_inc_meteo = pd.merge(df_incidentes, 
                  df_meteo,
                  left_on=['Fecha'],
                  right_on=['Fecha'],
                  how='left')

In [8]:
df_inc_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663588 entries, 0 to 663587
Data columns (total 41 columns):
Fecha               663588 non-null object
IdAhogado           663588 non-null int64
IdPersona           663588 non-null int64
Localidad           663588 non-null object
Provincia           663588 non-null object
CCAA                663588 non-null object
Hora                536825 non-null object
Latitud             663588 non-null float64
Longitud            663588 non-null float64
Sexo                663519 non-null object
Edad                541888 non-null float64
Nacionalidad        663588 non-null object
Origen              663588 non-null object
Extraccion          663588 non-null object
Titular             663588 non-null object
Causa               663588 non-null object
TipoAhogamiento     663588 non-null object
Factor              663588 non-null object
Intervencion        663588 non-null object
Pronostico          663588 non-null object
Localizacion        663588 n

In [9]:
def calcular_distancia(lon1,lat1,lon2,lat2):

    R = 6373.0

    dlon = lon2 - lon1

    dlat = lat2 - lat1

    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2

    c = 2 * atan2(sqrt(a), sqrt(1-a))

    distance = R * c

    #print ("Result", str(distance))

    return distance

In [10]:
df_inc_meteo['incremental'] = df_inc_meteo.index

In [11]:
df_inc_meteo.head()

,Fecha,IdAhogado,IdPersona,Localidad,Provincia,CCAA,Hora,Latitud,Longitud,Sexo,...,T_max,T_min,Precipitaciones,Presion,Dir. vi.,V_Viento,Nub.,Prof. nieve (cm),Insolac. (horas),incremental
0,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,16.4,8.4,0.0,1030.8 Hpa,164º(S),7,2/8,-,8.4,0
1,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,15.9,3.7,0.0,1030.8 Hpa,165º(S),9,-,-,8.2,1
2,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,14.8,10.9,0.0,1028.2 Hpa,258º(W),11,-,-,-,2
3,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,13.4,-2.4,0.0,1033.3 Hpa,143º(SE),3,0/8,-,5.5,3
4,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,15.6,5.8,0.0,1030.5 Hpa,238º(SW),13,2/8,-,8.5,4


In [12]:
df_inc_meteo_calculaDistancia = dict()

for num_linea, j in df_inc_meteo.iterrows():
    
    incremental=j[41]
    
    id_persona=j[2]
    
    fecha=j[0]

    i_longitud=j[8]

    i_latitud=j[7]
    
    id_estacion=j[26]
    
    e_longitud=j[30]

    e_latitud=j[29]

    #rint(num_linea)
    
    e_distancia=calcular_distancia(i_longitud,i_latitud,e_longitud,e_latitud)

    #rint("ID AHOGADO: " + str(id_persona) +" , Fecha: " + str(fecha)+ " , LONGITUD: "  + str(i_longitud) + " , LATITUD: " + str(i_latitud) + " , id_estacion: " + str(id_estacion) + " , LONGITUD: "  + str(e_longitud) + " , LATITUD: " + str(e_latitud))

    distancia = e_distancia
    
    #rint("Distancia: "+ str(distancia))
    
    estacion_number = id_estacion

    df_inc_meteo_calculaDistancia[incremental]=fecha,id_persona,estacion_number,distancia

 

df_inc_meteo_calculaDistancia

{0: ('30/12/2019', 6985, 8001, 930.7333175237114),
 1: ('30/12/2019', 6985, 8002, 927.3444537875805),
 2: ('30/12/2019', 6985, 8004, 878.3714890036248),
 3: ('30/12/2019', 6985, 8008, 850.1556114756938),
 4: ('30/12/2019', 6985, 8011, 743.0163423868682),
 5: ('30/12/2019', 6985, 8014, 716.9036886652041),
 6: ('30/12/2019', 6985, 8015, 725.963988527075),
 7: ('30/12/2019', 6985, 8016, 638.4635962663265),
 8: ('30/12/2019', 6985, 8021, 565.4999992708711),
 9: ('30/12/2019', 6985, 8023, 566.426101826601),
 10: ('30/12/2019', 6985, 8025, 492.6536035207035),
 11: ('30/12/2019', 6985, 8027, 424.2229812856095),
 12: ('30/12/2019', 6985, 8029, 406.99264741252),
 13: ('30/12/2019', 6985, 8032, 583.0449954234696),
 14: ('30/12/2019', 6985, 8036, 449.3409214139592),
 15: ('30/12/2019', 6985, 8037, 438.1101326995919),
 16: ('30/12/2019', 6985, 8042, 928.4043888701816),
 17: ('30/12/2019', 6985, 8044, 942.6576622103031),
 18: ('30/12/2019', 6985, 8045, 944.6817599620658),
 19: ('30/12/2019', 6985, 

In [13]:
df_distancias=pd.DataFrame.from_dict(df_inc_meteo_calculaDistancia, orient='index')

df_distancias.columns = ['fecha','IdPersona','ID_Estacion','Distancia']

In [14]:
df_distancias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663588 entries, 0 to 663587
Data columns (total 4 columns):
fecha          663588 non-null object
IdPersona      663588 non-null int64
ID_Estacion    663588 non-null int64
Distancia      663588 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 25.3+ MB


In [15]:
df_inc_meteo_distancia = pd.merge(df_inc_meteo, 
                  df_distancias,
                  left_on=['Fecha','IdPersona','ID'],
                  right_on=['fecha','IdPersona','ID_Estacion'],
                  how='left')

In [16]:
df_inc_meteo_distancia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663588 entries, 0 to 663587
Data columns (total 45 columns):
Fecha               663588 non-null object
IdAhogado           663588 non-null int64
IdPersona           663588 non-null int64
Localidad           663588 non-null object
Provincia           663588 non-null object
CCAA                663588 non-null object
Hora                536825 non-null object
Latitud             663588 non-null float64
Longitud            663588 non-null float64
Sexo                663519 non-null object
Edad                541888 non-null float64
Nacionalidad        663588 non-null object
Origen              663588 non-null object
Extraccion          663588 non-null object
Titular             663588 non-null object
Causa               663588 non-null object
TipoAhogamiento     663588 non-null object
Factor              663588 non-null object
Intervencion        663588 non-null object
Pronostico          663588 non-null object
Localizacion        663588 n

In [17]:
df_inc_meteo_distancia=df_inc_meteo_distancia.sort_values(by=['IdPersona', 'Distancia','Fecha'])

In [18]:
df_inc_meteo_distancia.head()

,Fecha,IdAhogado,IdPersona,Localidad,Provincia,CCAA,Hora,Latitud,Longitud,Sexo,...,Presion,Dir. vi.,V_Viento,Nub.,Prof. nieve (cm),Insolac. (horas),incremental,fecha,ID_Estacion,Distancia
663581,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1022.7 Hpa,340º(N),3,6/8,-,1.2,663581,01/01/2013,8419,47.562655
663586,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1021.2 Hpa,288º(W),17,6/8,-,5.3,663586,01/01/2013,8482,67.801181
663580,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1023.1 Hpa,58º(NE),2,7/8,-,0.0,663580,01/01/2013,8410,80.952206
663579,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1022.0 Hpa,74º(E),8,6/8,-,3.2,663579,01/01/2013,8397,116.278371
663578,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1022.8 Hpa,44º(NE),5,7/8,-,4.7,663578,01/01/2013,8391,141.546413


In [19]:
df_inc_meteo_distancia=df_inc_meteo_distancia.drop_duplicates(subset=['Fecha','IdAhogado','IdPersona','Localidad','Provincia','CCAA','Hora','Latitud','Longitud','Sexo','Edad','Nacionalidad','Origen','Extraccion','Titular','Causa','TipoAhogamiento','Factor','Intervencion','Pronostico','Localizacion','Riesgo','Reanimacion','Vigilancia','Actividad','Deteccion'],keep='first')

In [20]:
df_inc_meteo_distancia.head(99)

,Fecha,IdAhogado,IdPersona,Localidad,Provincia,CCAA,Hora,Latitud,Longitud,Sexo,...,Presion,Dir. vi.,V_Viento,Nub.,Prof. nieve (cm),Insolac. (horas),incremental,fecha,ID_Estacion,Distancia
663581,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.075250,Hombre,...,1022.7 Hpa,340º(N),3,6/8,-,1.2,663581,01/01/2013,8419,47.562655
663360,09/01/2013,146,168,PUERTO DE LA CRUZ,TENERIFE,CANARIAS,15:36,0.495824,-0.288784,Hombre,...,1028.7 Hpa,347º(N),5,1/8,-,7.9,663360,09/01/2013,8451,1349.441373
663528,05/01/2013,147,169,ARRECIFE,LANZAROTE,CANARIAS,16:00,0.504845,-0.237980,Hombre,...,1033.9 Hpa,350º(N),3,0/8,-,-,663528,05/01/2013,8451,1120.630572
663472,05/01/2013,148,170,SAN BARTOLOME DE TIRAJANA,GRAN CANARIA,CANARIAS,15:35,0.484508,-0.271814,Hombre,...,1033.9 Hpa,350º(N),3,0/8,-,-,663472,05/01/2013,8451,1339.953401
663416,06/01/2013,150,172,PLAYA DEL INGLES,GRAN CANARIA,CANARIAS,15:35,0.484488,-0.271640,Hombre,...,1030.9 Hpa,315º(NW),2,1/8,-,-,663416,06/01/2013,8451,1339.428023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641301,25/10/2013,552,290,PUERTO DE LA CRUZ,TENERIFE,CANARIAS,17:20,0.495791,-0.288883,Hombre,...,1011.6 Hpa,139º(SE),12,8/8,-,NaN,641301,25/10/2013,8383,1334.662499
641244,26/10/2013,553,291,"PLAYA DEL MATORRAL, MORRO JABLE, PAJARA",FUERTEVENTURA,CANARIAS,12:09,0.489495,-0.250114,Hombre,...,1019.1 Hpa,136º(SE),5,5/8,-,NaN,641244,26/10/2013,8451,1239.671312
641177,30/10/2013,554,292,"PLAYA CHICA, CORRALEJO, LA OLIVA",FUERTEVENTURA,CANARIAS,20:00,0.501584,-0.242032,Hombre,...,1023.1 Hpa,349º(N),12,3/8,10.1,NaN,641177,30/10/2013,8383,1149.656293
641107,31/10/2013,555,293,"PLAYA DEL PONIENTE, BENIDORM MUERE AHOGADO UN ...",ALICANTE,CDAD VALENCIANA,16:40,0.672586,-0.002543,Hombre,...,1021.9 Hpa,310º(NW),6,6/8,9.1,NaN,641107,31/10/2013,8359,36.173799


In [21]:
df_inc_meteo_distancia=df_inc_meteo_distancia.drop(['fecha', 'incremental','Titular','ID_Estacion'], axis=1)
df_inc_meteo_distancia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6482 entries, 663581 to 60
Data columns (total 41 columns):
Fecha               6482 non-null object
IdAhogado           6482 non-null int64
IdPersona           6482 non-null int64
Localidad           6482 non-null object
Provincia           6482 non-null object
CCAA                6482 non-null object
Hora                5381 non-null object
Latitud             6482 non-null float64
Longitud            6482 non-null float64
Sexo                6481 non-null object
Edad                5366 non-null float64
Nacionalidad        6482 non-null object
Origen              6482 non-null object
Extraccion          6482 non-null object
Causa               6482 non-null object
TipoAhogamiento     6482 non-null object
Factor              6482 non-null object
Intervencion        6482 non-null object
Pronostico          6482 non-null object
Localizacion        6482 non-null object
Riesgo              6482 non-null object
Reanimacion         6482 non

In [22]:
import re
df_inc_meteo_distancia['Presion'].replace(' Hpa','', inplace=True, regex=True)
df_inc_meteo_distancia['Precipitaciones'].replace('Ip','0', inplace=True, regex=True)
#df_inc_meteo_distancia['Presion'].replace(re.escape(' m(850)'),'850', inplace=True, regex=True)
#df_inc_meteo_distancia['Presion'].replace(re.escape(' m(925)'),'925', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(NE)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace('º','', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(NE)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(NW)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(N)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(S)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(E)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(W)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(SE)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace(re.escape('(SW)'),'', inplace=True, regex=True)
df_inc_meteo_distancia['Dir. vi.'].replace({'-': None},inplace =True, regex= True)
df_inc_meteo_distancia['Dir. vi.'].replace({'¼': '.25'},inplace =True, regex= True)
df_inc_meteo_distancia['Nub.'].replace({'-': None},inplace =True, regex= True)
df_inc_meteo_distancia['Prof. nieve (cm)'].replace({'-': None},inplace =True, regex= True)
df_inc_meteo_distancia['Insolac. (horas)'].replace({'-': None},inplace =True, regex= True)

In [23]:
df_inc_meteo_distancia=df_inc_meteo_distancia.rename(columns={"Nub.": "Nubosidad",
                                                              "Estado_x": "Estado",
                                                              "Longitud_y": "Longitud_est",
                                                              "Latitud_y": "Latitud_est",
                                                              "Longitud": "Longitud_inc",
                                                              "Latitud": "Latitud_inc",
                                                              "Dir. vi": "DirViento",
                                                              "Prof. nieve (cm)": "ProfNievecm",
                                                              "Insolac. (horas)": "InsolacHoras"})


In [24]:
df_inc_meteo_distancia[['Nub_nominador','Nub_denominador']]=df_inc_meteo_distancia.Nubosidad.str.split("/",expand=True)

In [25]:
df_inc_meteo_distancia.Nub_nominador.unique()

array(['6', '1', '0', '8', '7', '5', '4', '2', '3', None], dtype=object)

In [26]:
df_inc_meteo_distancia.Nub_nominador.fillna(value='9999', inplace=True)
df_inc_meteo_distancia.Nub_denominador.fillna(value='9999', inplace=True)
df_inc_meteo_distancia.ProfNievecm.fillna(value='9999', inplace=True)
df_inc_meteo_distancia.InsolacHoras.fillna(value='9999', inplace=True)

In [27]:
df_inc_meteo_distancia.Nub_nominador.unique()

array(['6', '1', '0', '8', '7', '5', '4', '2', '3', '9999'], dtype=object)

In [28]:
df_inc_meteo_distancia['Nub_nominador'] = df_inc_meteo_distancia['Nub_nominador'].astype(int)
df_inc_meteo_distancia['Nub_denominador'] = df_inc_meteo_distancia['Nub_denominador'].astype(int)

df_inc_meteo_distancia['Nub_denominador'] = df_inc_meteo_distancia['Nub_denominador'].astype(int)
df_inc_meteo_distancia['Nub_denominador'] = df_inc_meteo_distancia['Nub_denominador'].astype(int)

In [29]:
df_inc_meteo_distancia['ProfNievecm'] = df_inc_meteo_distancia.Nub_nominador.replace(9999, np.nan)
df_inc_meteo_distancia['InsolacHoras'] = df_inc_meteo_distancia.Nub_denominador.replace(9999, np.nan)

In [30]:
df_inc_meteo_distancia['Nubosidad']=df_inc_meteo_distancia['Nub_nominador']/df_inc_meteo_distancia['Nub_denominador']

In [31]:
df_inc_meteo_distancia=df_inc_meteo_distancia.drop(['Nub_nominador','Nub_denominador'], axis=1)

In [32]:
temp=df_inc_meteo_distancia.Presion

In [33]:
df_inc_meteo_distancia['Presion'] = pd.np.where(temp.str.contains("850") == True ,"850",df_inc_meteo_distancia['Presion'])

In [34]:
df_inc_meteo_distancia['Presion'] = pd.np.where(temp.str.contains("925") == True ,"925",df_inc_meteo_distancia['Presion'])

In [35]:
df_inc_meteo_distancia.Presion.unique()

array(['1022.7', '1028.7', '1033.9', '1030.9', '1018.9', '1004.3',
       '1016.8', '1016.0', '1017.0', '1017.7', '1013.7', '850', '1022.9',
       '1020.8', '1019.7', '1012.4', '1018.8', '1017.9', '1018.5',
       '1021.4', '1026.0', '1020.2', '1020.0', '1020.9', '1021.0',
       '1019.4', '1012.2', '1020.4', '1020.7', '1015.8', '1016.2',
       '1015.6', '1014.7', '1014.5', '1014.9', '1018.3', '1013.3',
       '1012.9', '1014.3', '1015.0', '1023.9', '1018.1', '1019.0',
       '1015.7', '1012.6', '1016.7', '1025.4', '1014.2', '1013.4',
       '1014.6', '925', '1016.5', '1021.2', '1015.3', '1017.6', '1017.3',
       '1019.8', '1015.9', '1010.9', '1008.2', '1018.6', '1013.0',
       '1011.6', '1019.1', '1023.1', '1021.9', '1024.4', '1024.1',
       '1022.2', '1020.1', '1014.1', '1009.2', '1025.2', '1027.6',
       '1024.3', '1027.0', '1022.1', '1016.3', '1024.0', '1033.1',
       '1021.3', '1013.8', '1019.5', '1018.2', '1019.2', '1027.2',
       '1027.1', '1032.2', '1028.9', '1024.8', '

In [36]:
df_inc_meteo_distancia.to_csv('../DATASETS/AHOGADOS/CSV/historico_incidentes_estaciones_meteo.csv',sep='|',header=True,index=False)